In [3]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report
import re
import string


In [4]:
data_fake=pd.read_csv('Fake.csv')
data_true=pd.read_csv('True.csv')

In [5]:
data_fake.head(5)
data_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
data_fake['class']=0
data_true['class']=1


In [7]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [8]:
data_fake_manuel_testing=data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i],axis=0,inplace=True)

In [9]:
data_true_manuel_testing=data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i],axis=0,inplace=True)

In [10]:
data_fake.shape,data_true.shape

((23471, 5), (21407, 5))

In [11]:
data_merge=pd.concat([data_fake,data_true],axis=0)
data_merge.tail(10)

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


In [12]:
data=data_merge.drop(['title','subject','date'],axis=1)

In [13]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
data=data.sample(frac=1)


In [15]:
data.reset_index(inplace=True)
data.drop(['index'],axis=1,inplace=True)

In [16]:
data.head(2)

,text,class
0,Everyone knows by now just how fast and loose ...,0
1,Democratic party leaders from four states have...,0


In [17]:
def wordopt(text):
    text=text.lower()
    text=re.sub('\[.*?]','',text)
    text=re.sub('\\W',' ',text)
    text=re.sub('<.*?>+',' ',text)
    text=re.sub('https?//\s+[www\.\s+]','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [18]:
x='Amal is a  king?/*'
wordopt(x)

'amal is a  king   '

In [19]:
data['text']=data['text'].apply(wordopt)

In [20]:
data.head(10)

,text,class
0,everyone knows by now just how fast and loose ...,0
1,democratic party leaders from four states have...,0
2,warning this video is graphic,0
3,donald trump kicked off his monday morning by ...,0
4,tokyo reuters japan is hammering out plans...,1
5,bill o reilly was dismissed overnight from fox...,0
6,beirut reuters the syrian government welco...,1
7,berlin reuters germany s conservative chan...,1
8,washington reuters republican senator and ...,1
9,msnbc host thomas roberts on sunday was specul...,0


In [21]:
x=data['text']
y=data['class']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer 
import pickle
vectorisation=TfidfVectorizer()
xv_train=vectorisation.fit_transform(x_train)
xv_test=vectorisation.transform(x_test)
pickle.dump(vectorisation, open("vectorisation.pkl",'wb'))

In [24]:
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [26]:
pred_lr=LR.predict(xv_test)
pred_lr

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [27]:
LR.score(xv_test,y_test)

0.986096256684492

In [28]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5812
           1       0.99      0.99      0.99      5408

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [29]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [30]:
pred_dt=DT.predict(xv_test)

In [31]:
DT.score(xv_test,y_test)

0.9950089126559715

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
GB=GradientBoostingClassifier()
GB.fit(xv_train,y_train)

GradientBoostingClassifier()

In [33]:
predict_gb=GB.predict(xv_test)

In [34]:
GB.score(xv_test,y_test)

0.9959893048128342

In [35]:
import pickle

In [36]:
pickle.dump(GB,open('gbmodel.pkl','wb'))

In [37]:
pickle.dump(DT,open('dtmodel.pkl','wb'))

In [38]:
pickle.dump(LR,open('lrmodel.pkl','wb'))

In [39]:
def manual_testing(news):
    testing_news={'text':[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test['text']=new_def_test['text'].apply(wordopt)
    new_x_test=new_def_test['text']
    new_xv_test=vectorisation.transform(new_x_test)
    pred_GB=GB.predict(new_xv_test)
    return pred_GB

manual_testing(""" Trump Said Some INSANELY Racist Stuff Inside The Oval Office, And Witnesses Back It Up","In the wake of yet another court decision that derailed Donald Trump s plan to bar Muslims from entering the United States, the New York Times published a report on Saturday morning detailing the president s frustration at not getting his way   and how far back that frustration goes.According to the article, back in June, Trump stomped into the Oval Office, furious about the state of the travel ban, which he thought would be implemented and fully in place by then. Instead, he fumed, visas had already been issued to immigrants at such a rate that his  friends were calling to say he looked like a fool  after making his broad pronouncements.It was then that Trump began reading from a document that a top advisor, noted white supremacist Stephen Miller, had handed him just before the meeting with his Cabinet. The page listed how many visas had been issued this year, and included 2,500 from Afghanistan (a country not on the travel ban), 15,000 from Haiti (also not included), and 40,000 from Nigeria (sensing a pattern yet?), and Trump expressed his dismay at each.According to witnesses in the room who spoke to the Times on condition of anonymity, and who were interviewed along with three dozen others for the article, Trump called out each country for its faults as he read: Afghanistan was a  terrorist haven,  the people of Nigeria would  never go back to their huts once they saw the glory of America, and immigrants from Haiti  all have AIDS. Despite the extensive research done by the newspaper, the White House of course denies that any such language was used.But given Trump s racist history and his advisor Stephen Miller s blatant white nationalism, it would be no surprise if a Freedom of Information Act request turned up that the document in question had the statements printed inline as commentary for the president to punctuate his anger with. It was Miller, after all, who was responsible for the  American Carnage  speech that Trump delivered at his inauguration.This racist is a menace to America, and he doesn t represent anything that this country stands for. Let s hope that more indictments from Robert Mueller are on their way as we speak.Featured image via Chris Kleponis/Pool/Getty Images",News,"December 23, 2017""")


array([0], dtype=int64)